In [7]:
import sys
sys.path.append('..')

import os
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

In [8]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_SHP_RECORRIDOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\RECORRIDOS.shp'
GDF_RECORRIDOS = gpd.read_file(PATH_SHP_RECORRIDOS)

In [9]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_parte_diario_sin_cod_labor():
    engine = obtener_engine()
    try:
        query = """
            SELECT DISTINCT idd, unidad_01, unidad_02
            FROM drones_pulverizacion.parte_diario_pulv 
            WHERE idd IS NOT NULL AND cod_labor IS NULL
        """
        # Usar pandas si no hay geometría involucrada
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return pd.DataFrame()

def obtener_propiedad_por_idd(idd):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv WHERE idd= {idd}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def actualizar_area_rociada(id, val):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    area_rocia= :val
                WHERE id = :id
            """)
            conn.execute(query, {"id": id, "val": val})
            print(f"✔️ Area rociada actualizada de lote id: {id}")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

def actualizar_faltantes_por_idd(idd, fecha_intermedia, fecha_min, fecha_max, numero_semana, num_vuelos, total_flight_time, total_caudal):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    fecha= :fecha_intermedia,
                    hora_ini= :fecha_min,
                    hora_fin= :fecha_max,
                    semana= :numero_semana,
                    num_vuelos= :num_vuelos,
                    horas_vuel= :total_flight_time,
                    total_caud= :total_caudal
                WHERE idd = :idd
            """)
            conn.execute(query, {
                "idd": idd, 
                "fecha_intermedia": fecha_intermedia,
                "fecha_min": fecha_min,
                "fecha_max": fecha_max,
                "numero_semana": numero_semana,
                "num_vuelos": num_vuelos,
                "total_flight_time": total_flight_time,
                "total_caudal": total_caudal,
            })
            print(f"✔️ Datos faltantes de idd: {idd} cargados correctamente")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

In [10]:
propiedades_procesar = obtener_parte_diario_sin_cod_labor()
propiedades_procesar

,idd,unidad_01,unidad_02
0,64,275,SAN JOSE--CATALA


In [11]:
propiedades_procesar

,idd,unidad_01,unidad_02
0,64,275,SAN JOSE--CATALA


In [12]:
for i, row in propiedades_procesar.iterrows():
    idd = row.idd
    selec_recorridos = GDF_RECORRIDOS[GDF_RECORRIDOS['idd']==idd]
    # selccion de lotes planificados
    selec_plan = obtener_propiedad_por_idd(idd)
    # conviente en campo fecha a formato fecha
    selec_recorridos.loc[:, "fecha"] = pd.to_datetime(selec_recorridos["fecha"])
    # Obtener la fecha mínima y máxima
    fecha_min = selec_recorridos["fecha"].min()
    fecha_max = selec_recorridos["fecha"].max()
    fecha_intermedia = fecha_min + (fecha_max - fecha_min) / 2
    
    numero_semana = fecha_intermedia.isocalendar().week
    area_total_pulv = selec_recorridos['area'].sum()
    num_vuelos = len(selec_recorridos)
    total_caudal = selec_recorridos['spray'].sum()
    
    selec_recorridos = selec_recorridos.copy()
    
    # obtener horas de vuelo en formto 00:00:00
    selec_recorridos["fl_time_sec"] = selec_recorridos["fl_time"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
    # Sum all seconds
    total_seconds = selec_recorridos["fl_time_sec"].sum()
    # Convert back to hh:mm:ss format
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    total_flight_time = f"{hours:02}:{minutes:02}:{seconds:02}"  # 00:00:00

    # Aplicar el buffer a la geometría de los recorridod
    selec_recorridos["geometry"] = selec_recorridos.geometry.buffer(5.5)
    # Disolver todas las geometrías en un solo polígono
    buffer_dissolved = selec_recorridos.dissolve()
    # Calcular la intersección
    interseccion = gpd.overlay(buffer_dissolved, selec_plan, how="intersection")
    # Calcular el área de cada polígono de la intersección
    interseccion["area_rociado"] = interseccion.geometry.area / 10000
    # seleccion solo de la columnas de interes
    resumen = interseccion[['id_2', 'unidad_05', 'area_2', 'area_rociado']].copy()
    
    # PROCEDIMIENTO PARA AJUSTAR EL AREA DE ROCIADO
    suma_area_rociada = resumen['area_rociado'].sum()
    area_rociada_real = area_total_pulv  # aquí pon tu valor real
    # Factor inicial de ajuste
    factor_ajuste = area_rociada_real / suma_area_rociada
    # Ajuste proporcional inicial
    resumen['area_ajustada'] = resumen['area_rociado'] * factor_ajuste
    # Verificar que no exceda area_2 y corregir iterativamente si es necesario
    while any(resumen['area_ajustada'] > resumen['area_2']):
        exceso = resumen['area_ajustada'] > resumen['area_2']
        resumen.loc[exceso, 'area_ajustada'] = resumen.loc[exceso, 'area_2']
        # Recalcular factor solo con las filas no ajustadas al máximo permitido
        area_ajustada_parcial = resumen.loc[~exceso, 'area_ajustada'].sum()
        area_restante = area_rociada_real - resumen.loc[exceso, 'area_ajustada'].sum()
        # Evitar división por cero
        if area_ajustada_parcial == 0:
            break
        nuevo_factor = area_restante / area_ajustada_parcial
        resumen.loc[~exceso, 'area_ajustada'] *= nuevo_factor
    # calcular campos faltantes
    resumen["porcen_rociado"] = (resumen["area_ajustada"]) / (resumen["area_2"])
    resumen["dif_area"] = resumen["area_2"] - resumen["area_ajustada"]
    # recorrer resumen para actualizar area rociada en el parte diario
    for i, row in resumen.iterrows():
        print(row.id_2, row.area_ajustada)
        actualizar_area_rociada(row.id_2, row.area_ajustada)
    # cargar campos restantes del parte con respecto a idd
    actualizar_faltantes_por_idd(idd, fecha_intermedia, fecha_min, fecha_max, numero_semana, num_vuelos, total_flight_time, total_caudal)

449 4.532334966835828
✔️ Area rociada actualizada de lote id: 449
483 3.1285595453115844
✔️ Area rociada actualizada de lote id: 483
434 3.459552362293306
✔️ Area rociada actualizada de lote id: 434
435 0.6741412408664049
✔️ Area rociada actualizada de lote id: 435
436 3.32273409879414
✔️ Area rociada actualizada de lote id: 436
437 1.7668506879270798
✔️ Area rociada actualizada de lote id: 437
438 9.72127693531103
✔️ Area rociada actualizada de lote id: 438
439 8.497388082627689
✔️ Area rociada actualizada de lote id: 439
442 1.8302706441652314
✔️ Area rociada actualizada de lote id: 442
447 1.1452884759758348
✔️ Area rociada actualizada de lote id: 447
459 3.4057419954803887
✔️ Area rociada actualizada de lote id: 459
462 4.488584457793156
✔️ Area rociada actualizada de lote id: 462
463 5.967696871437836
✔️ Area rociada actualizada de lote id: 463
453 3.9764734117116167
✔️ Area rociada actualizada de lote id: 453
452 4.387478970883763
✔️ Area rociada actualizada de lote id: 452
455 5